In [149]:
import spacy
from spacy import displacy

NER = spacy.load("en_core_web_sm")

In [150]:
#installed libraries

!pip install gTTS
!pip install playsound  
!pip install pyttsx3  
!apt install libasound2-dev portaudio19-dev libportaudio2 libportaudiocpp0 ffmpeg
!pip install speechRecognition
!pip install pydub
!pip install flashtext

'apt' is not recognized as an internal or external command,
operable program or batch file.


In [151]:
#imports

import numpy as np
import re
import nltk
from sklearn.datasets import load_files
nltk.download('stopwords')
import pickle
from nltk.corpus import stopwords
import gtts  
from playsound import playsound  
import pyttsx3
import os
import speech_recognition as sr
import pyaudio
import wave
import IPython.display as ipd
from IPython.display import Audio
from gtts import gTTS
import time
import librosa
import soundfile
import os, glob, pickle
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score
from pydub import AudioSegment
from flashtext import KeywordProcessor

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\iamro\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [152]:
def output_speech(mytext):
    language = 'en'
    myobj = gTTS(text=mytext, lang=language, slow=False)
    myobj.save("chat_speech.mp3")
    os.system("chat_speech.mp3")
    print("RECRUITER:",mytext)

In [153]:
def input_speech():
    CHUNK = 1024 
    FORMAT = pyaudio.paInt16 #paInt8
    CHANNELS = 2 
    RATE = 44100 #sample rate
    RECORD_SECONDS = 10
    WAVE_OUTPUT_FILENAME = "input.wav"

    p = pyaudio.PyAudio()

    stream = p.open(format=FORMAT,
                channels=CHANNELS,
                rate=RATE,
                input=True,
                frames_per_buffer=CHUNK) #buffer

    print("* recording")

    frames = []

    for i in range(0, int(RATE / CHUNK * RECORD_SECONDS)):
        data = stream.read(CHUNK)
        frames.append(data) # 2 bytes(16 bits) per channel

    print("* done recording")

    stream.stop_stream()
    stream.close()
    p.terminate()

    wf = wave.open(WAVE_OUTPUT_FILENAME, 'wb')
    wf.setnchannels(CHANNELS)
    wf.setsampwidth(p.get_sample_size(FORMAT))
    wf.setframerate(RATE)
    wf.writeframes(b''.join(frames))
    wf.close()
    sound = AudioSegment.from_wav("input.wav")
    sound = sound.set_channels(1)
    sound.export("input.wav", format="wav")
    r = sr.Recognizer()
    with sr.WavFile("input.wav") as source:
        audio_data = r.listen(source)
        # convert speech to text
        text = r.recognize_google(audio_data)
        print("ME:",text)
    sentiment = predict_sentiment(text)
    return audio_data, predict_emo('input.wav')-sentiment

In [154]:
def place_info(P):
    place_d = {'Hyderabad':"it's famous for it's biryani !" }
    return place_d[P]

In [155]:
def extract_feature(file_name, mfcc, chroma, mel):
    with soundfile.SoundFile(file_name) as sound_file:
        X = sound_file.read(dtype = "float32")
        sample_rate = sound_file.samplerate
        if chroma:
            stft = np.abs(librosa.stft(X))
        result = np.array([])
        if mfcc:
            mfccs = np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T, axis=0)
            result = np.hstack((result, mfccs))
        if chroma:
            chroma = np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T,axis=0)
            result = np.hstack((result, chroma))
        if mel:
            mel = np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)
            result = np.hstack((result, mel))
    return result

In [156]:
emotions = {
  '01':'neutral',
  '02':'calm',
  '03':'happy',
  '04':'sad',
  '05':'angry',
  '06':'fearful',
  '07':'disgust',
  '08':'surprised' }

observed_emotions = ['calm', 'happy', 'fearful', 'disgust']
def load_data(test_size=0.2):
    x,y = [],[]
    for file in glob.glob("D:\datasets\SER\Actor_*\\*.wav"):
        file_name = os.path.basename(file)
        emotion = emotions[file_name.split("-")[2]]
        if emotion not in observed_emotions:
            continue
        feature = extract_feature(file, mfcc=True, chroma=True, mel=True)
        x.append(feature)
        y.append(emotion)
    return train_test_split(np.array(x), y, test_size=test_size, random_state=101)

In [157]:
X_train, X_test, y_train, y_test = load_data(test_size=0.25)

In [158]:
model = MLPClassifier(alpha=0.01, batch_size=256, epsilon=1e-08, 
                      hidden_layer_sizes=(300,200,100), learning_rate='adaptive', max_iter=500)
model.fit(X_train,y_train)

MLPClassifier(alpha=0.01, batch_size=256, hidden_layer_sizes=(300, 200, 100),
              learning_rate='adaptive', max_iter=500)

In [159]:
model

MLPClassifier(alpha=0.01, batch_size=256, hidden_layer_sizes=(300, 200, 100),
              learning_rate='adaptive', max_iter=500)

In [160]:
y_pred=model.predict(X_test)

In [161]:
accuracy=accuracy_score(y_true=y_test, y_pred=y_pred)
print("Accuracy: {:.2f}%".format(accuracy*100))

Accuracy: 50.52%


In [162]:
def predict_emo(file):
    x = []
    feature = extract_feature(file, mfcc=True, chroma=True, mel=True)
    x.append(feature)
    np.array(x)
    l = model.predict(x)
    s = ''
    s.join(l)
    dec = -100
    if s in ['sad','fearful']:
        dec = 100
    if s in ['disgust','angry']:
        dec = 200
    return dec

In [163]:
place_info("Hyderabad")

"it's famous for it's biryani !"

In [197]:

def personal_round():
    r = sr.Recognizer()
    points_scored = 1000 #points keep decreasing based on answers by the candidate
    
    #question 1
    mytext = "Hello, let's start with some personal details about you. What is your name, where are you from and where did u work earlier?"
    language = 'en'
    output_speech(mytext)
    time.sleep(11)
    audio_data, emote_points = input_speech()  
    points_scored -= int(emote_points)
    text1 = NER(r.recognize_google(audio_data))
    for word in text1.ents:
        if word.label_=="PERSON":
            name=word.text
        if word.label_=="GPE":
            place=word.text
        if word.label_=="ORG":
            prev=word.text
    N="Rishabh"        
    mytext = "Hi " + N + ", all the best for your interview!"
    output_speech(mytext)
    time.sleep(5)
    
    mytext = N + ", What are some of your strengths?"
    output_speech(mytext)
    time.sleep(6)
    audio_data, emote_points = input_speech()
    points_scored += emote_points
    strenghts = ['enthusiasm','enthusiastic','trustworthiness','trustworthy','creativity','creative','discipline','disciplined','patience','patient','respectfulness','respectfull','determination','determined','dedication','dedicated','honesty','honest','versatility','versatile','liveliness','lively','hard working']
    keywordprocessor = KeywordProcessor(case_sensitive=False)
    keywordprocessor.add_keywords_from_list(strenghts)
    Extractedkeywords = keywordprocessor.extract_keywords(r.recognize_google(audio_data))
    points_scored += len(keywordprocessor)*100
    
    mytext = N + ", What are some of your weaknesses?"
    output_speech(mytext)
    time.sleep(6)
    audio_data, emote_points = input_speech()
    points_scored += emote_points
    keywordprocessor = KeywordProcessor(case_sensitive=False)
    keywordprocessor.add_keywords_from_list(strenghts)
    Extractedkeywords = keywordprocessor.extract_keywords(r.recognize_google(audio_data))
    points_scored -= len(keywordprocessor)*50
    
    P = "Hyderabad"
    mytext = "Tell me about " + P + ",how is it there and what is " + P + "famous for." + " Heard " + place_info(place)
    output_speech(mytext)
    time.sleep(6)
    audio_data, emote_points = input_speech()
    points_scored += emote_points
    
    prev = "Samsung"
    mytext = "Why did you leave from " + prev
    output_speech(mytext)
    time.sleep(6)
    audio_data, emote_points = input_speech()
    points_scored += emote_points
    
    mytext = "Your score till now is" + str(points_scored)
    output_speech(mytext)
    time.sleep(6)
    
    mytext = "Let's move to some technical questions!"
    output_speech(mytext)
    time.sleep(6)


SyntaxError: invalid syntax (Temp/ipykernel_12680/3215681510.py, line 53)

In [165]:
#resume summarizer.

In [166]:
def technical_round(points_scored):
    
    
    #question 1
    mytext = "What is a doubly linked list and specify its applications?"
    output_speech(mytext)
    time.sleep(8)
    audio_data, emote_points = input_speech()
    points_scored += emote_points
    #answer 1
    answer = "This is a complex type of a linked list wherein a node has two references: One that connects to the next node in the sequence Another that connects to the previous node. This structure allows traversal of the data elements in both directions (left to right and vice versa). Applications of DLL are: A music playlist with next song and previous song navigation options. The browser cache with BACK-FORWARD visited pages The undo and redo functionality on platforms such as word, paint etc, where you can reverse the node to get to the previous page."
    
    #question 2
    mytext = "What is a priority queue?"
    output_speech(mytext)
    time.sleep(8)
    audio_data, emote_points = input_speech()
    points_scored += emote_points
    #answer 2
    answer = " A priority queue is an abstract data type that is like a normal queue but has priority assigned to elements. Elements with higher priority are processed before the elements with a lower priority. In order to implement this, a minimum of two queues are required - one for the data and the other to store the priority."
    
    #question 3
    mytext = "What is a AVL tree?"
    output_speech(mytext)
    time.sleep(8)
    audio_data, emote_points = input_speech()
    points_scored += emote_points
    #answer 3
    answer = "AVL trees are height balancing BST. AVL tree checks the height of left and right sub-trees and assures that the difference is not more than 1. This difference is called Balance Factor and is calculates as. BalanceFactor = height(left subtree) − height(right subtree)"
    
    #question 4
    mytext = "What is a heap?"
    output_speech(mytext)
    time.sleep(8)
    audio_data, emote_points = input_speech()
    points_scored += emote_points
    #answer 4
    answer = "Heap is a special tree-based non-linear data structure in which the tree is a complete binary tree. A binary tree is said to be complete if all levels are completely filled except possibly the last level and the last level has all elements towards as left as possible."
    
    #question 5
    mytext = "What is the difference between BFS and DFS?"
    output_speech(mytext)
    time.sleep(8)
    audio_data, emote_points = input_speech()
    points_scored += emote_points
    #answer 5
    answer = "BFS and DFS both are the traversing methods for a graph. Graph traversal is nothing but the process of visiting all the nodes of the graph. The main difference between BFS and DFS is that BFS traverses level by level whereas DFS follows first a path from the starting to the end node, then another path from the start to end, and so on until all nodes are visited. Furthermore, BFS uses queue data structure for storing the nodes whereas DFS uses the stack for traversal of the nodes for implementation. DFS yields deeper solutions that are not optimal, but it works well when the solution is dense whereas the solutions of BFS are optimal. You can learn more about BFS here: Breadth First Search and DFS here: Depth First Search."
    
    #question 6
    mytext = "What is your work experience such as internship and freelancing?"
    output_speech(mytext)
    time.sleep(8)
    audio_data, emote_points = input_speech()
    points_scored += emote_points
    #answer 5
    companies_pos = ["apple","samsung","foxconn","alphabet", "microsoft" ,"huawei","dell","hitachi", "amazon","flipkart","atlassian","google","oracle","fiverr","fiver"]
    companies_neg = ["no","sorry","not","couldn't"]
    keywordprocessor = KeywordProcessor(case_sensitive=False)
    keywordprocessor.add_keywords_from_list(companies_pos)
    Extractedkeywords = keywordprocessor.extract_keywords(r.recognize_google(audio_data))
    points_scored += len(keywordprocessor)*50
    keywordprocessor = KeywordProcessor(case_sensitive=False)
    keywordprocessor.add_keywords_from_list(companies_neg)
    Extractedkeywords = keywordprocessor.extract_keywords(r.recognize_google(audio_data))
    points_scored -= len(keywordprocessor)*50
    

In [193]:
import re
import matplotlib.pyplot as plt
import string
from nltk.corpus import stopwords
import nltk
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
from nltk.tokenize.treebank import TreebankWordDetokenizer
from collections import Counter
#from wordcloud import WordCloud
from nltk.corpus import stopwords
import nltk
from gensim.utils import simple_preprocess
from nltk.corpus import stopwords
import gensim
from sklearn.model_selection import train_test_split
import spacy
import pickle
import warnings
warnings.filterwarnings('ignore')
import seaborn as sns
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt 
import tensorflow as tf
import keras
import numpy as np
import pandas as pd

In [168]:
train = pd.read_csv('D:/datasets/sentiment analysis/train.csv')
train.head(15)

,textID,text,selected_text,sentiment
0,cb774db0d1,"I`d have responded, if I were going","I`d have responded, if I were going",neutral
1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,negative
2,088c60f138,my boss is bullying me...,bullying me,negative
3,9642c003ef,what interview! leave me alone,leave me alone,negative
4,358bd9e861,"Sons of ****, why couldn`t they put them on t...","Sons of ****,",negative
5,28b57f3990,http://www.dothebouncy.com/smf - some shameles...,http://www.dothebouncy.com/smf - some shameles...,neutral
6,6e0c6d75b1,2am feedings for the baby are fun when he is a...,fun,positive
7,50e14c0bb8,Soooo high,Soooo high,neutral
8,e050245fbd,Both of you,Both of you,neutral
9,fc2cbefa9d,Journey!? Wow... u just became cooler. hehe....,Wow... u just became cooler.,positive


In [169]:
len(train)

27481

In [170]:
train['sentiment'].unique()

array(['neutral', 'negative', 'positive'], dtype=object)

In [171]:
train = train[['selected_text','sentiment']]
train["selected_text"].fillna("No content", inplace = True)

In [172]:
def depure_data(data):
    
    #Removing URLs with a regular expression
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    data = url_pattern.sub(r'', data)

    # Remove Emails
    data = re.sub('\S*@\S*\s?', '', data)

    # Remove new line characters
    data = re.sub('\s+', ' ', data)

    # Remove distracting single quotes
    data = re.sub("\'", "", data)
        
    return data

In [173]:
temp = []
#Splitting pd.Series to list
data_to_list = train['selected_text'].values.tolist()
for i in range(len(data_to_list)):
    temp.append(depure_data(data_to_list[i]))
list(temp[:5])

['I`d have responded, if I were going',
 'Sooo SAD',
 'bullying me',
 'leave me alone',
 'Sons of ****,']

In [174]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True)) 
        # deacc=True removes punctuations
        

data_words = list(sent_to_words(temp))

print(data_words[:10])

[['have', 'responded', 'if', 'were', 'going'], ['sooo', 'sad'], ['bullying', 'me'], ['leave', 'me', 'alone'], ['sons', 'of'], ['some', 'shameless', 'plugging', 'for', 'the', 'best', 'rangers', 'forum', 'on', 'earth'], ['fun'], ['soooo', 'high'], ['both', 'of', 'you'], ['wow', 'just', 'became', 'cooler']]


In [175]:
def detokenize(text):
    return TreebankWordDetokenizer().detokenize(text)

In [176]:
data = []
for i in range(len(data_words)):
    data.append(detokenize(data_words[i]))
print(data[:5])

['have responded if were going', 'sooo sad', 'bullying me', 'leave me alone', 'sons of']


In [177]:
data = np.array(data)

In [178]:
labels = np.array(train['sentiment'])
y = []
for i in range(len(labels)):
    if labels[i] == 'neutral':
        y.append(0)
    if labels[i] == 'negative':
        y.append(1)
    if labels[i] == 'positive':
        y.append(2)
y = np.array(y)
labels = tf.keras.utils.to_categorical(y, 3, dtype="float32")
del y

In [179]:
from keras.models import Sequential
from keras import layers
from tensorflow.keras.optimizers import RMSprop,Adam
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras import regularizers
from keras import backend as K
from keras.callbacks import ModelCheckpoint
max_words = 5000
max_len = 200

tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(data)
sequences = tokenizer.texts_to_sequences(data)
tweets = pad_sequences(sequences, maxlen=max_len)
print(tweets)

[[   0    0    0 ...   68  146   41]
 [   0    0    0 ...    0  397   65]
 [   0    0    0 ...    0    0   11]
 ...
 [   0    0    0 ...  372   10    3]
 [   0    0    0 ...   24  542    4]
 [   0    0    0 ... 2424  199  657]]


In [180]:
X_train, X_test, y_train, y_test = train_test_split(tweets,labels, random_state=0)
print (len(X_train),len(X_test),len(y_train),len(y_test))

20610 6871 20610 6871


In [181]:
model2 = Sequential()
model2.add(layers.Embedding(max_words, 40, input_length=max_len))
model2.add(layers.Bidirectional(layers.LSTM(20,dropout=0.6)))
model2.add(layers.Dense(3,activation='softmax'))
model2.compile(optimizer='rmsprop',loss='categorical_crossentropy', metrics=['accuracy'])
#Implementing model checkpoins to save the best metric and do not lose it on training.
checkpoint2 = ModelCheckpoint("best_model2.hdf5", monitor='val_accuracy', verbose=1,save_best_only=True, mode='auto', period=1,save_weights_only=False)
history = model2.fit(X_train, y_train, epochs=5,validation_data=(X_test, y_test),callbacks=[checkpoint2])

Epoch 1/5
645/645 [==============================] - ETA: 0s - loss: 0.7929 - accuracy: 0.6482 E
Epoch 00001: val_accuracy improved from -inf to 0.73294, saving model to best_model2.hdf5
645/645 [==============================] - 68s 99ms/step - loss: 0.7929 - accuracy: 0.6482 - val_loss: 0.6404 - val_accuracy: 0.7329
Epoch 2/5
645/645 [==============================] - ETA: 0s - loss: 0.5803 - accuracy: 0.7676
Epoch 00002: val_accuracy improved from 0.73294 to 0.79101, saving model to best_model2.hdf5
645/645 [==============================] - 58s 90ms/step - loss: 0.5803 - accuracy: 0.7676 - val_loss: 0.5306 - val_accuracy: 0.7910
Epoch 3/5
645/645 [==============================] - ETA: 0s - loss: 0.5094 - accuracy: 0.7985
Epoch 00003: val_accuracy improved from 0.79101 to 0.80512, saving model to best_model2.hdf5
645/645 [==============================] - 61s 95ms/step - loss: 0.5094 - accuracy: 0.7985 - val_loss: 0.5066 - val_accuracy: 0.8051
Epoch 4/5
645/645 [===================

In [182]:
best_model = keras.models.load_model("best_model2.hdf5")
test_loss, test_acc = best_model.evaluate(X_test, y_test, verbose=2)
print('Model accuracy: ',test_acc)

215/215 - 5s - loss: 0.4681 - accuracy: 0.8189 - 5s/epoch - 24ms/step
Model accuracy:  0.8189492225646973


In [183]:
predictions = best_model.predict(X_test)

In [184]:
predictions

array([[0.00522882, 0.01172866, 0.9830425 ],
       [0.017721  , 0.9753108 , 0.00696822],
       [0.5261967 , 0.01086186, 0.46294138],
       ...,
       [0.03248564, 0.9484236 , 0.01909068],
       [0.92924863, 0.02571947, 0.0450319 ],
       [0.55078554, 0.41869769, 0.03051668]], dtype=float32)

In [185]:
X_test.shape

(6871, 200)

In [186]:
def predict_sentiment(x):
    #Removing URLs with a regular expression
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    x = url_pattern.sub(r'', x)
    # Remove Emails
    x = re.sub('\S*@\S*\s?', '', x)
    # Remove new line characters
    x = re.sub('\s+', ' ', x)
    # Remove distracting single quotes
    x = re.sub("\'", "", x)
    
    
    temp=[]
    temp.append(x)
    data_words = list(sent_to_words(temp))
    
    data = []
    for i in range(len(data_words)):
        data.append(detokenize(data_words[i]))
    
    data = np.array(data)
    max_words = 5000
    max_len = 200
    
    tokenizer = Tokenizer(num_words=max_words)
    tokenizer.fit_on_texts(data)
    sequences = tokenizer.texts_to_sequences(data)
    num_val = pad_sequences(sequences, maxlen=max_len)
    
    y = best_model.predict(num_val)
    if max(y[0]) == y[0][0]:
        return 25
    if max(y[0]) == y[0][1]:
        return -50
    if max(y[0]) == y[0][2]:
        return 50

In [187]:
predict_sentiment("I`d have responded, if I were going")

25

In [188]:
import gensim
from gensim.models import KeyedVectors
import os



In [190]:
sentence_1 = "Obama speaks to the media in Illinois"
sentence_2 = "President greets the press in Chicago"
sentence_3 = "Nokia is my favorite company"

s1='my name is rishabh'
s2='rishabh is the name'



In [ ]:
personal_round()